In [ ]:
import os

from edf.pc_utils import draw_geometry, voxel_filter
from edf.data import PointCloud, SE3, TargetPoseDemo, DemoSequence, DemoSeqDataset
from edf.preprocess import Rescale, NormalizeColor, Downsample
from edf.agent import PickAgent, PlaceAgent

import numpy as np
import yaml
import plotly as pl
import plotly.express as ple
import open3d as o3d

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose

torch.set_printoptions(precision= 3, sci_mode=False, linewidth=120)

In [ ]:
device = 'cuda:0'
# device = 'cpu'
unit_len = 0.01

load_transforms = Compose([Rescale(rescale_factor=1/unit_len),
                          ])
trainset = DemoSeqDataset(dataset_dir="demo/test_demo", annotation_file="data.yaml", load_transforms = load_transforms, device=device)
# train_dataloader = DataLoader(trainset, shuffle=False, collate_fn=lambda xs:{'processed': [x['processed'] for x in xs], 'raw': [x['raw'] for x in xs]}) 
train_dataloader = DataLoader(trainset, shuffle=False, collate_fn=lambda x:x)

In [ ]:
place_agent_config_dir = "config/agent_config/place_agent_dev.yaml"
place_agent_param_dir = "checkpoint/mug_1_demo_dev/place/model_iter_300.pt"
max_N_query_place = 3
langevin_dt_place = 0.001

place_agent = PlaceAgent(config_dir=place_agent_config_dir, 
                       device = device,
                       max_N_query = max_N_query_place, 
                       langevin_dt = langevin_dt_place)

place_agent.load(place_agent_param_dir, strict=False)

In [ ]:
scene_proc_fn = Compose([Downsample(voxel_size=1.7, coord_reduction="average"),
                            NormalizeColor(color_mean = torch.tensor([0.5, 0.5, 0.5]), color_std = torch.tensor([0.5, 0.5, 0.5]))])
grasp_proc_fn = Compose([Downsample(voxel_size=1.4, coord_reduction="average"),
                            NormalizeColor(color_mean = torch.tensor([0.5, 0.5, 0.5]), color_std = torch.tensor([0.5, 0.5, 0.5]))])

In [ ]:
for train_batch in train_dataloader:
    for data in train_batch:
        demo_seq_raw: DemoSequence = data
        break
    break

In [ ]:
scene_raw = demo_seq_raw[1].scene_pc
grasp_raw = demo_seq_raw[1].grasp_pc
scene_proc = scene_proc_fn(scene_raw)
grasp_proc = grasp_proc_fn(grasp_raw)

In [ ]:
# query_points_visual = []
# for query_point in pick_agent.query_model.query_points.cpu():
#     mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.5)
#     mesh_sphere.compute_vertex_normals()
#     mesh_sphere.paint_uniform_color([0.7, 0.1, 0.1])
#     mesh_sphere.translate(query_point)
#     query_points_visual.append(mesh_sphere)

# draw_geometry([grasp_pc_raw] + query_points_visual)

In [ ]:
T_seed = 100
place_policy = 'sorted'
place_mh_iter = 1000
place_langevin_iter = 300
place_dist_temp = 1.
place_policy_temp = 1.
place_optim_iter = 100
place_optim_lr = 0.005
place_query_temp = 1.

In [ ]:
Ts, edf_outputs, logs = place_agent.forward(scene=scene_proc, T_seed=T_seed, grasp=grasp_proc, policy = place_policy, mh_iter=place_mh_iter, langevin_iter=place_langevin_iter, 
                                            temperature=place_dist_temp, policy_temperature=place_policy_temp, optim_iter=place_optim_iter, optim_lr=place_optim_lr, query_temperature=place_query_temp)

In [ ]:
grasps = grasp_raw.transformed(Ts[1].to(device))

In [ ]:
draw_geometry([scene_proc] + [grasps])